# Автоматическая обработка текстов 
## Домашнее задание 1 [20 баллов] до 23:59 28.04.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из двух частей:
1. Генерация текста по шаблону
2. Извлечение телефонных номеров из текста.

## Часть 1 [10 баллов]
Задание состоит из следующих пунктов:
1. Скачивание данных о состоянии погоды в городе 
2. Генерация описания прогноза
3. Творческая часть

Все три пункта можно считать независимыми – вы можете сделать один или два из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 

### 1. Сбор данных [3 балла]


Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание, с любого сервиса с прогнозом погоды или используя его API.
Примеры сервисов:
* gismeteo.ru, https://www.gismeteo.ru/api/ – Gismeteo, API Gismeteo 
* https://tech.yandex.ru/weather/ – API Yandex.Погоды
* https://sinoptik.com.ru – Sinoptik
* любой другой 

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальная температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

In [3]:
import requests

r = requests.get("http://api.wunderground.com/api/f58c032a9510345a/forecast10day/q/Russia/Moscow.json")
print(r.status_code)
response_data = r.json()

200


In [11]:
data = response_data['forecast']['simpleforecast']['forecastday']

In [46]:
from datetime import datetime
import math
import locale

locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

class Weather(object):
    def __init__(self, epoch, low, high, wind, precip):
        self.dt = datetime.fromtimestamp(int(epoch))
        self.low = float(low)
        self.high = float(high)
        self.wind = float(wind)
        self.precip = float(precip)
        
    def __repr__(self):
        return f"{self.dt} temp: {self.low}..{self.high} wind: {self.wind}m/s precip: {self.precip}mm"

forecast = list()
for day in data:
    w = Weather(
        epoch=day['date']['epoch'],
        low=day['low']['celsius'],
        high=day['high']['celsius'],
        wind=math.ceil(day['maxwind']['kph']*1000.0/3600),
        precip=day['qpf_allday']['mm']
    )
    forecast.append(w)
    
forecast


[2018-04-27 19:00:00 temp: 4.0..15.0 wind: 2.0m/s precip: 0.0mm,
 2018-04-28 19:00:00 temp: 5.0..15.0 wind: 9.0m/s precip: 0.0mm,
 2018-04-29 19:00:00 temp: 8.0..16.0 wind: 7.0m/s precip: 0.0mm,
 2018-04-30 19:00:00 temp: 11.0..21.0 wind: 7.0m/s precip: 4.0mm,
 2018-05-01 19:00:00 temp: 11.0..24.0 wind: 9.0m/s precip: 5.0mm,
 2018-05-02 19:00:00 temp: 10.0..19.0 wind: 7.0m/s precip: 0.0mm,
 2018-05-03 19:00:00 temp: 8.0..18.0 wind: 7.0m/s precip: 0.0mm,
 2018-05-04 19:00:00 temp: 9.0..19.0 wind: 7.0m/s precip: 8.0mm,
 2018-05-05 19:00:00 temp: 8.0..18.0 wind: 5.0m/s precip: 1.0mm,
 2018-05-06 19:00:00 temp: 8.0..17.0 wind: 7.0m/s precip: 0.0mm]

In [47]:
import pandas as pd

In [48]:
index = [w.dt.strftime("%d.%m (%a)") for w in forecast]
d = {
    'минимальная температура': [w.low for w in forecast],
    'максимальная температура': [w.high for w in forecast],
    'скорость ветра': [w.wind for w in forecast],
    'уровень осадков': [w.precip for w in forecast]
}

f = pd.DataFrame(d, index=index)

In [49]:
f.T

,27.04 (пт),28.04 (сб),29.04 (вс),30.04 (пн),01.05 (вт),02.05 (ср),03.05 (чт),04.05 (пт),05.05 (сб),06.05 (вс)
максимальная температура,15.0,15.0,16.0,21.0,24.0,19.0,18.0,19.0,18.0,17.0
минимальная температура,4.0,5.0,8.0,11.0,11.0,10.0,8.0,9.0,8.0,8.0
скорость ветра,2.0,9.0,7.0,7.0,9.0,7.0,7.0,7.0,5.0,7.0
уровень осадков,0.0,0.0,0.0,4.0,5.0,0.0,0.0,8.0,1.0,0.0


### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на первые 10 дней февраля в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 км/час в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 мм за 7 дней*
    * *Выпадет 10 см снега за ближайшие 7 дней * 


Вместо НазваниеГорода и дней недели подставьте название выбранного вами города и дни недели, используя фунцкии для согласования существительных с предлогами. Используйте функции для согласования числительного с существительным для согласования длительности промежутков времени и слова "день" и чисел  с последующими единицами измерения.

In [37]:
import pymorphy2
morph = pymorphy2.analyzer.MorphAnalyzer()
city = 'Москва'

In [65]:
def forecast_one(city, day1, day2, forecast):
    # В _четверг_ в _Городе_ потеплеет на _7_ градусов по сравнению со _средой_
    
    day1w = forecast[day1].dt.strftime("%A")
    prep1 = 'В' if day1w != 'вторник' else 'Во'
    day1w = morph.parse(day1w)[0].inflect({'accs'}).word
    
    day2w = forecast[day2].dt.strftime("%A")
    prep2 = 'с' if day2w != 'среда' else 'со'
    day2w = morph.parse(day2w)[0].inflect({'ablt'}).word
    
    city = morph.parse(city)[0].inflect({'loct'}).word

    if forecast[day1].high == forecast[day2].high:
        return f'{prep1} {day1w} в {city[0].upper()+city[1:]} погода не изменится по сравнению {prep2} {day2w}.'
    
    change = 'потеплеет' if forecast[day1].high > forecast[day2].high else 'похолодает'
    amount = math.ceil(abs(forecast[day1].high - forecast[day2].high))
    degree = morph.parse('градус')[0].make_agree_with_number(int(amount)).word
    
    return f'{prep1} {day1w} в {city[0].upper()+city[1:]} {change} на {amount} {degree} по сравнению {prep2} {day2w}.'

for i in range(9):
    print(forecast_one('Москва', i+1, i, forecast))

В субботу в Москве погода не изменится по сравнению с пятницей.
В воскресенье в Москве потеплеет на 1 градус по сравнению с субботой.
В понедельник в Москве потеплеет на 5 градусов по сравнению с воскресеньем.
Во вторник в Москве потеплеет на 3 градуса по сравнению с понедельником.
В среду в Москве похолодает на 5 градусов по сравнению с вторником.
В четверг в Москве похолодает на 1 градус по сравнению со средой.
В пятницу в Москве потеплеет на 1 градус по сравнению с четвергом.
В субботу в Москве похолодает на 1 градус по сравнению с пятницей.
В воскресенье в Москве похолодает на 1 градус по сравнению с субботой.


In [68]:
def forecast_two(city, day1, day2, forecast):
    # Скорость ветра изменится на 3 км/час в понедельник по сравнению с пятницей
    
    day1w = forecast[day1].dt.strftime("%A")
    prep1 = 'в' if day1w != 'вторник' else 'Во'
    day1w = morph.parse(day1w)[0].inflect({'accs'}).word
    
    day2w = forecast[day2].dt.strftime("%A")
    prep2 = 'с' if day2w != 'среда' else 'со'
    day2w = morph.parse(day2w)[0].inflect({'ablt'}).word

    if forecast[day1].wind == forecast[day2].wind:
        return f'Скорость ветра не изменится {prep1} {day1w} по сравнению {prep2} {day2w}.'
    
    change = 'увеличится' if forecast[day1].wind > forecast[day2].wind else 'уменьшится'
    amount = math.ceil(abs(forecast[day1].wind - forecast[day2].wind))
    
    return f'Скорость ветра {change} на {amount} м/с {prep1} {day1w} по сравнению {prep2} {day2w}.'

for i in range(9):
    print(forecast_two('Москва', i+1, i, forecast))

Скорость ветра увеличится на 7 м/с в субботу по сравнению с пятницей.
Скорость ветра уменьшится на 2 м/с в воскресенье по сравнению с субботой.
Скорость ветра не изменится в понедельник по сравнению с воскресеньем.
Скорость ветра увеличится на 2 м/с Во вторник по сравнению с понедельником.
Скорость ветра уменьшится на 2 м/с в среду по сравнению с вторником.
Скорость ветра не изменится в четверг по сравнению со средой.
Скорость ветра не изменится в пятницу по сравнению с четвергом.
Скорость ветра уменьшится на 2 м/с в субботу по сравнению с пятницей.
Скорость ветра увеличится на 2 м/с в воскресенье по сравнению с субботой.


In [76]:
def forecast_three(city, day1, day2, forecast):
    # Уровень осадков понизится на 3.85 мм за 7 дней
    
    interval = day1 - day2
    days = morph.parse('дни')[0].make_agree_with_number(interval).word
    
    if forecast[day1].precip == forecast[day2].precip:
        return f'Уровень осадков не изменится за {interval} {days}.'
    
    change = 'повысится' if forecast[day1].wind > forecast[day2].wind else 'понизится'
    amount = abs(forecast[day1].wind - forecast[day2].wind)
    
    return f'Уровень осадков {change} на {amount} мм за {interval} {days}.'

for i in range(1,9):
    print(forecast_three('Москва', i, 0, forecast))

Уровень осадков не изменится за 1 день.
Уровень осадков не изменится за 2 дня.
Уровень осадков повысится на 5.0 мм за 3 дня.
Уровень осадков повысится на 7.0 мм за 4 дня.
Уровень осадков не изменится за 5 дней.
Уровень осадков не изменится за 6 дней.
Уровень осадков повысится на 5.0 мм за 7 дней.
Уровень осадков повысится на 3.0 мм за 8 дней.


### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

Генерировать текст по шаблонам может быть необходимо на любом веб-сайте, поисковой выдаче. Например, "по заданному условию найдено X документов(а)"

Текст об изменениях числовых показателей может понадобится для информирования о ходе биржевых торгов, спортивных соревнований и т.п. Везде, где есть поток числовых данных.

Можно использовать несколько шаблонов и чередовать их или выбирать в случайном порядке. Например, "Завтра в Москве похолодает на 3 градуса", "Завтра термометры покажут на 3 градуса меньше", "Завтра столбик термометра опустится на 3 градуса". 

Можно также попробовать менять порядок слов в предложении

## Часть 2 [10 баллов]
Данная часть состоит из пяти пунктов.
1. Генерация датасета.
2. Обучение классификатора.
3. Поиск номеров регулярным выражением.
4. Оценка качества работы.

### 1. Генерация датасета [2 балла].
Возьмите любой русский текст (можно взять твиты из семинара). Разбейте его на предложения. Вставьте <b>случайные</b> телефонные номера в любую часть предложения.

Примеры номеров:
>89268659970

>8(495)3451212

>+7(234)456-78-90

>8(956)234-23-23

>85679962312 

>+7-845-344-23-65

Можете придумать еще вид номеров. Чтобы сгенерировать случайный номер используйте Шаблон и вставляйте в него случайный номер. Запомните позицию вставки (номер слова, после которого будет вставлен номер).

In [ ]:
<your code here>

После того, как у вас получился набор предложений с номерами промаркируйте данные. Они должны иметь такой вид:

[("Уи́льям", 0), ("Шекспи́р", 0), ("—", 0), ("английский", 0), ("8(495)3451212", 1), ("поэт", 0), ("и", 0), ("драматург", 0)]

In [ ]:
<your code here>

Придумайте признаки для номера и сгенерируйте датасет.

In [ ]:
<your code here>

### 2. Обучение классификатора [3 балла].
Обучите один или несколько классификаторов.

In [ ]:
<your code here>

### 3. Поиск номеров регулярным выражением [3 балла].
Напишите регулярное выражение для извлечения номера из текста.

In [ ]:
<your code here>

### 4. Оценка качества работы [2 балла].

Охарактеризуйте результаты.

In [ ]:
<your code here>

In [ ]:
<your answer here>

Ответьте на вопросы:
* правильно ли мы генерировали данные для задачи?
* какие признаки и классификаторы будут работать лучше, если мы знаем, как были получены данные.

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 28.04.2018. 

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.


Сдача отчетов осуществляется по email: login-const@mail.ru, заголовок письма – NLP-HSE-HW1-ИмяФамилия. 
